# Retrieve Time-series data at a pixel.

Function to retrieve time series data at a given pixel from timeseries file in `.h5` format generated by Mintpy.

### Import libraries

In [1]:
from mintpy.objects import timeseries
from mintpy.utils import readfile, utils as ut #only needed if coordinates are in lat/long format
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import medfilt
import pandas as pd

### Function to extract time-series data at a given pixel with input coordinates in radar coordinates format.

In [ ]:
def getInSARtimeseries_yx( y, x, tsfile, startdate, enddate,  formatstr = '%Y%m%d', plot = False, outname = None):
    # input: y, x -- coordinates of target pixel
    # input: tsfile -- time-series file
    # input: startdate -- start date of the time-series
    # input: enddate -- end date of the time-series
    # input: plot -- plot or not?

    # read time-series file
    ts_obj = timeseries(tsfile)
    date_list = ts_obj.get_date_list()
    date = [pd.to_datetime(x, format='%Y%m%d') for x in date_list]

    startdate_int = pd.to_datetime(startdate, format=formatstr)
    enddate_int = pd.to_datetime(enddate, format=formatstr)
    
    # find the nearest dates
    date_diff = np.array([x-startdate_int for x in date])
    startdate_idx = np.argmin(abs(date_diff))
    date_diff = np.array([x-enddate_int for x in date])
    enddate_idx = np.argmin(abs(date_diff))
    
    # retrieve a box of time-series centered at y,x and average
    box = 10
    ts = ts_obj.read()
    ts = ts[startdate_idx:enddate_idx,y-box:y+box+1,x-box:x+box+1]*100 # convert unit to cm
    ts_avg = np.mean(ts, axis = (1,2))
    ts_avg = ts_avg-np.mean(ts_avg)
    if plot:
        ts_avg_m = medfilt(ts_avg, 1)
        ts_max = max(ts_avg_m)
        ts_min = min(ts_avg_m)
        vmax = max(abs(ts_max),abs(ts_min))
        opts = {'c': 'k', 'linewidth': 2}
        fig = plt.figure(figsize=[10,3])
        plt.scatter(date[startdate_idx:enddate_idx], ts_avg_m)
        plt.grid(True)
        plt.xlabel('time',fontsize=12)
        plt.ylabel('displacements, cm', fontsize=12)
        plt.yticks(fontsize=12)
        plt.xticks(fontsize=12)
        plt.ylim([-1.5*vmax,1.5*vmax])
        if outname is not None:
            
            plt.savefig(outname,format='png')
        plt.show()
    return date[startdate_idx:enddate_idx], ts_avg

### If coordinates are in latitude, longitude format

If the coordinates of the pixel are in latitude, longitude format, they can first be converted to radar coordinates using the following function.

In [ ]:
def latlon2yx(latlist,lonlist, velfile, geomfile):
    # input: latlist -- list of latitudes or single latitude
    # input: lonlist -- list of longitudes or single longitude
    # input: velfile -- velocity file generated by mintpy
    # input: geomfile -- geometry file generated by mintpy
    atr = readfile.read_attribute(velfile)
    coord = ut.coordinate(atr,geomfile)
    if type(latlist) is list:
        x=[]
        y=[]
        num = len(latlist)
        for i in range(num):
            ytmp,xtmp = coord.geo2radar(latlist[i],lonlist[i])[0:2]
            x.append(xtmp)
            y.append(ytmp)
    else:
        y,x = coord.geo2radar(latlist,lonlist)[0:2]

    return y, x

After converting to latitude, longitude format, we can then use the `getInSARtimeseries_yx` function to extract time-series data at the required pixel.